In [1]:
import uproot
import awkward as ak
from array import array
import sys, os
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt
from scipy.optimize import curve_fit
from matplotlib.ticker import AutoMinorLocator, MultipleLocator, FuncFormatter

# -----------------------------------------------------
# Settings
# -----------------------------------------------------

# Electrons
run_electron_number = 24242
beam_charge = 18122.907  # BCM4C Beam Cut Charge: 18122.907 uC
d_beam_charge = .001 # beam_charge_uncertainty in uC
live_time = 1
d_live_time = 0
tracking_eff = 0.9984 # E SING FID TRACK EFFIC         :     0.9984 +-   0.0001
d_tracking_eff = 0.0001 # tracking eff uncertainty
prescale_factor = 1 # Ps4_factor = 1
beam_charge_milliC = beam_charge / 1000  # mC
d_beam_charge_milliC = d_beam_charge / 1000 # converting to millicoulomb

# Positrons
run_positron_number = 24549
beam_charge_pos = 43022.260  # placeholder
live_time_pos = 1.0
tracking_eff_pos = 0.9947
prescale_factor_pos = 3.0
beam_charge_milliC_pos = beam_charge_pos / 1000  # mC

USE_LOG_SCALE = False  # Toggle log scale on/off for y-axis

# -----------------------------------------------------
# File paths
# -----------------------------------------------------

electron_file = f'/w/hallc-scshelf2102/c-rsidis/relder/ROOTfiles/hms_coin_replay_production_{run_electron_number}_-1.root'
positron_file = f'/w/hallc-scshelf2102/c-rsidis/relder/ROOTfiles/hms_coin_replay_production_{run_positron_number}_-1.root'

# -----------------------------------------------------
# Open files and access trees
# -----------------------------------------------------

def open_trees(file_tree_map):
    trees = {}
    for label, (file_path, tree_name) in file_tree_map.items():
        try:
            file = uproot.open(file_path)
            tree = file[tree_name]
            trees[label] = tree
            print(f"Successfully opened '{tree_name}' from: {file_path}")
        except Exception as e:
            print(f"Failed to open {tree_name} from {file_path}: {e}")
    return trees

file_tree_map = {
    "data_electron": (electron_file, "T"),
    "data_positron": (positron_file, "T")
}

trees = open_trees(file_tree_map)

# -----------------------------------------------------
# Load arrays
# -----------------------------------------------------

branches = [
    "H.gtr.dp", "H.cal.etottracknorm", "H.gtr.ph", "H.gtr.th",
    "H.gtr.x", "H.gtr.y", "H.kin.Q2", "H.kin.x_bj", "H.kin.W", "H.cer.npeSum"
]

electrons = trees["data_electron"].arrays(branches, library="np")
positrons = trees["data_positron"].arrays(branches, library="np")

# -----------------------------------
# Electrons
# -----------------------------------

delta = electrons['H.gtr.dp']
cer = electrons['H.cer.npeSum']
cal = electrons['H.cal.etottracknorm']

cut = (delta > -8) & (delta < 8) & (cer > 2) & (cal > 0.8)
electrons_cut = {key: arr[cut] for key, arr in electrons.items()}

weight_electrons = prescale_factor / (beam_charge_milliC * live_time * tracking_eff)
weight_factor_electrons = np.full_like(electrons_cut['H.gtr.dp'], weight_electrons, dtype=float)

# -----------------------------------
# Positrons
# -----------------------------------

delta_pos = positrons['H.gtr.dp']
cer_pos = positrons['H.cer.npeSum']
cal_pos = positrons['H.cal.etottracknorm']

cut_pos = (delta_pos > -8) & (delta_pos < 8) & (cer_pos > 2) & (cal_pos > 0.8)
positrons_cut = {key: arr[cut_pos] for key, arr in positrons.items()}

weight_positrons = 1
weight_factor_positrons = np.full_like(positrons_cut['H.gtr.dp'], weight_positrons, dtype=float)

# -----------------------------------
# Histogram settings
# -----------------------------------

data_bins_delta = np.linspace(-10, 10, 100)   # Delta
data_bins_theta = np.linspace(-0.1, 0.1, 100) # Theta/xptar
data_bins_W = np.linspace(2, 4, 100)          # W
data_bins_ytar = np.linspace(-5, 5, 100)       # ytar
data_bins_Q2 = np.linspace(0, 7, 100)          # Q2
data_bins_xbj = np.linspace(0, 0.625, 100)     # x_Bj

#-----------------------------------
# Weighted histogram and errors for Delta
#-----------------------------------

#-----------------------------------
# Delta histogram with proper weighted errors
#-----------------------------------

# Weighted histogram counts
elec_counts_weighted, _ = np.histogram(
    electrons_cut['H.gtr.dp'],
    bins=data_bins_delta,
    weights=weight_factor_electrons
)

# Statistical uncertainty per bin (weighted)
elec_stat_err = np.sqrt(
    np.histogram(
        electrons_cut['H.gtr.dp'],
        bins=data_bins_delta,
        weights=weight_factor_electrons**2
    )[0]
)

# Multiplicative scale uncertainties (relative)
rel_beam_charge = d_beam_charge_milliC / beam_charge_milliC
rel_live_time = d_live_time / live_time
rel_tracking = d_tracking_eff / tracking_eff

# Total uncertainty per bin (stat + multiplicative)
elec_err = np.sqrt(
    elec_stat_err**2 + 
    (elec_counts_weighted * rel_beam_charge)**2 +
    (elec_counts_weighted * rel_live_time)**2 +
    (elec_counts_weighted * rel_tracking)**2
)

# Bin centers for plotting
data_bins_delta_center = (data_bins_delta[1:] + data_bins_delta[:-1]) / 2

#-----------------------------------
# Calculate integrals for electron and positron data for each variable
#-----------------------------------

integrals = {}

integrals['Delta_e'] = integrate_hist(electrons_cut['H.gtr.dp'], weight_factor_electrons, data_bins_delta)
integrals['Delta_pos'] = integrate_hist(positrons_cut['H.gtr.dp'], weight_factor_positrons, data_bins_delta)

integrals['Theta_e'] = integrate_hist(electrons_cut['H.gtr.th'], weight_factor_electrons, data_bins_theta)
integrals['Theta_pos'] = integrate_hist(positrons_cut['H.gtr.th'], weight_factor_positrons, data_bins_theta)

integrals['W_e'] = integrate_hist(electrons_cut['H.kin.W'], weight_factor_electrons, data_bins_W)
integrals['W_pos'] = integrate_hist(positrons_cut['H.kin.W'], weight_factor_positrons, data_bins_W)

integrals['ytar_e'] = integrate_hist(electrons_cut['H.gtr.y'], weight_factor_electrons, data_bins_ytar)
integrals['ytar_pos'] = integrate_hist(positrons_cut['H.gtr.y'], weight_factor_positrons, data_bins_ytar)

integrals['Q2_e'] = integrate_hist(electrons_cut['H.kin.Q2'], weight_factor_electrons, data_bins_Q2)
integrals['Q2_pos'] = integrate_hist(positrons_cut['H.kin.Q2'], weight_factor_positrons, data_bins_Q2)

integrals['xbj_e'] = integrate_hist(electrons_cut['H.kin.x_bj'], weight_factor_electrons, data_bins_xbj)
integrals['xbj_pos'] = integrate_hist(positrons_cut['H.kin.x_bj'], weight_factor_positrons, data_bins_xbj)

# Calculate ratios positron/electron
ratios = {}
for var in ['Delta', 'Theta', 'W', 'ytar', 'Q2', 'xbj']:
    e_int = integrals[f'{var}_e']
    p_int = integrals[f'{var}_pos']
    ratio = p_int / e_int if e_int != 0 else np.nan
    ratios[var] = ratio
    print(f'Integral ratio Positron/Electron for {var}: {ratio:.4f}')

#-----------------------------------
# Plotting
#-----------------------------------

fig, axes = plt.subplots(3, 2, figsize=(12, 12))

# Delta block with weighted error bars
# Plot
axes[0,0].hist(
    electrons_cut['H.gtr.dp'],
    bins=data_bins_delta,
    weights=weight_factor_electrons,
    histtype='step',
    color='red',
    label='Electrons'
)

# Plot error bars with hollow markers
axes[0,0].errorbar(
    data_bins_delta_center,
    elec_counts_weighted,
    yerr=elec_err,
    fmt='o',            # hollow circle marker
    mfc='none',         # facecolor none = hollow
    mec='red',          # marker edge color
    ecolor='red',
    capsize=3,
    label='elec'
)

axes[0,0].set_xlabel('Delta', fontsize=15)
axes[0,0].set_ylabel('Counts', fontsize=15)
axes[0,0].set_title('Delta', fontsize=20)
axes[0,0].legend()
axes[0,0].grid(alpha=0.5)
axes[0,0].text(
    0.05, 0.85, f'Ratio Pos/Elec = {ratios["Delta"]:.3f}',
    transform=axes[0,0].transAxes, fontsize=12,
    color='black', bbox=dict(facecolor='white', alpha=0.7)
)


# All other plots (Theta, W, ytar, Q2, xbj) remain as in your original script
# ... (copy the same plotting code from your existing script for these) ...

# applying the log scale toggle
if USE_LOG_SCALE:
    for ax in axes.flat:
        ax.set_yscale('log')

plt.tight_layout()
#plt.savefig('Electron_Positron_Plots.pdf')
plt.show()

Successfully opened 'T' from: /w/hallc-scshelf2102/c-rsidis/relder/ROOTfiles/hms_coin_replay_production_24242_-1.root
Successfully opened 'T' from: /w/hallc-scshelf2102/c-rsidis/relder/ROOTfiles/hms_coin_replay_production_24549_-1.root


NameError: name 'integrate_hist' is not defined